In [7]:
from tkinter import * 
from tkinter import messagebox
from tkinter import filedialog
import cv2
import shutil
import os
import numpy as np
import pandas as pd
from PIL import Image, ImageTk
from tkinter import simpledialog
from openpyxl import load_workbook

import time
import datetime
import string 
import pickle
import random

win=Tk()
win.state("zoomed")
win.resizable(width=False,height=False)
win.configure(bg="orange")
win.title("My project")

lbl_title=Label(win,text="E-Attendance Monitoting System",font=('',55,'bold','underline'),bg='orange')
lbl_title.pack()



#----------------------------------------Image---------------------------------------
imageFrame=None
import cv2
import pickle
from PIL import Image, ImageTk

imageFrame=None
import cv2
import pickle
from PIL import Image, ImageTk

imageFrame=None
def detect_faces_in_image(file_path):
    # initialize the face recognizer
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read("classifier.xml")

    # initialize the face cascade classifier
    harcascadePath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(harcascadePath)

    # create a dataframe to store the results
    df = pd.DataFrame(columns=["Name", "Date", "Time"])

    # load the image
    im = cv2.imread(file_path)

    # convert the image to grayscale
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    # detect faces in the image
    faces = faceCascade.detectMultiScale(gray, 1.2, 5)

    # process each detected face
    for (x, y, w, h) in faces:
        cv2.rectangle(im, (x,y), (x+w,y+h), (225,0,0), 2)
        Id, conf = recognizer.predict(gray[y:y+h,x:x+w])

        if conf >= 80:
            # get the name corresponding to the ID from the Excel file
            data = pd.read_excel("Results.xlsx")
            name = data.loc[data["ID"] == Id, "Name"].values[0]

            cv2.putText(im, "Recognized: " + str(name), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

            
        else:   
            cv2.putText(im, "Unknown", (x,y+h), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

# display the image with detected faces
    cv2.imshow("Detected Faces", im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

def browse():
    global imageFrame
    if(imageFrame!=None):
        imageFrame.destroy()
    file_path=filedialog.askopenfilename()
    frame=cv2.imread(file_path)
    cv2image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    
    imageFrame= Frame(width=700, height=500,bd=6,bg='black')
    lmain =Label(imageFrame)
    lmain.grid(row=0, column=0)
    imageFrame.place(relx=.37,rely=.3)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    lmain.imgtk = imgtk
    lmain.configure(image=imgtk)
    btn_Detection=Button(command=lambda:detect_faces_in_image(file_path),text='Detect face',font=('',20,'bold'),bd=10,width=12).place(relx=.4,rely=.9)

def image_screen():
    frm=Frame(win,bg='sky blue')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
 
    btn_Browse=Button(frm,command=lambda:browse(),text="Browse",font=('',20,'bold'),bd=10,width=8)
    btn_Browse.place(relx=.38,rely=.01)

    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)
#----------------------------------------------xxxxxxxxxx-----------------------------------------------------------    


#-------------------------------------------Video_Screen--------------------------------------------------------------
iFrame=None
def browse_video():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read("classifier.xml")
   
    harcascadePath = "haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(harcascadePath)
    file_path=filedialog.askopenfilename()
    frame=cv2.imread(file_path) 
    cam = cv2.VideoCapture(file_path)
    cv2.FONT_HERSHEY_SIMPLEX
    txt = ""
    
    # create a dataframe to store the results
    df = pd.DataFrame(columns=["Name", "ID", "Date", "Time"])
    global Name,Id,Date
    while True:
        ret, im = cam.read()
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray, 1.2, 5)
    
        for (x, y, w, h) in faces:
            cv2.rectangle(im, (x,y), (x+w,y+h), (225,0,0), 2)
            Id, conf = recognizer.predict(gray[y:y+h,x:x+w])
            
            if conf >= 50:
                # get the name corresponding to the ID from the Excel file
                data = pd.read_excel("Results.xlsx")
                name = data.loc[data["ID"] == Id, "Name"].values[0]
                
                cv2.putText(im, "Recognized: " + str(name), (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                
                # add the result to the dataframe
                df = df.append({"Name": name, "ID": Id, "Date": datetime.date.today().strftime("%Y-%m-%d"), "Time": datetime.datetime.now().strftime("%H:%M:%S")}, ignore_index=True)
                
            else:   
                noOfFile = len(os.listdir("ImagesUnknown"))+1
                #cv2.imwrite("ImagesUnknown\Image" + str(noOfFile) + ".jpg", im[y:y+h,x:x+w])
                Id='Unknown'
                txt=str(Id)
                cv2.putText(im, str(txt), (x,y+h), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                
            ts = time.time()
            date = datetime.datetime.fromtimestamp(ts).strftime("%Y-%m-%d")
            timeStamp = datetime.datetime.fromtimestamp(ts).strftime("%H:%M:%S")
        
        cv2.imshow('ret',im)
        
        if (cv2.waitKey(1) == ord('q')):
            cam.release()
            cv2.destroyAllWindows()
            
    
def video_screen(): 
    frm=Frame(win,bg='sky blue')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
  
    btn_Browse=Button(frm,command=lambda:browse_video(),text="Browse",font=('',20,'bold'),bd=10,width=8)
    btn_Browse.place(relx=.37,rely=.05)
    btn_back=Button(frm,command=lambda:Welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)
  
   
#--------------------------------------------WebCam-----------------------------------------------------------------
flag=False  
clf=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def stop():
    global lmain,cap,lmain,imageFrame
    cap.release()
    lmain.destroy()
    imageFrame.destroy()

def startfac():
    global flag
    flag=True
    
def stopface():
    global flag
    
    flag=False

def generate_timestamp():
        return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
import pandas as pd

def show_frame():
    recognizer = cv2.face.LBPHFaceRecognizer_create()
    recognizer.read("classifier.xml")
   
    harcascadePath = "D:\Project\haarcascade_frontalface_default.xml"
    faceCascade = cv2.CascadeClassifier(harcascadePath)
     
    cam = cv2.VideoCapture(0)
    cv2.FONT_HERSHEY_SIMPLEX
    txt=""
    
    df = pd.DataFrame(columns=['Id', 'Name', 'Time In', 'Time Out']) # create a Pandas DataFrame to store the data
    
    while True:
        ret, im = cam.read()
        gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
        faces = faceCascade.detectMultiScale(gray, 1.2, 5)
        
        for (x, y, w, h) in faces:
            cv2.rectangle(im, (x,y), (x+w,y+h), (225,0,0), 2)
            Id, conf = recognizer.predict(gray[y:y+h,x:x+w])
            
            if conf >= 80:
                name = "Person " + str(Id)
                cv2.putText(im, "Recognized: " + name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
                cv2.imwrite("Images\Image" + str(Id) + ".jpg", im[y:y+h,x:x+w])
            else:   
                #noOfFile = len(os.listdir("ImagesUnknown"))+1
                #cv2.imwrite("ImagesUnknown\Image" + str(noOfFile) + ".jpg", im[y:y+h,x:x+w])
                name = 'Unknown'
                txt=str(name)
                cv2.putText(im, str(txt), (x,y+h), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
                
            ts = time.time()
            date = datetime.datetime.fromtimestamp(ts).strftime("%Y-%m-%d")
            timeStamp = datetime.datetime.fromtimestamp(ts).strftime("%H:%M:%S")
            
            if name not in df['Name'].values: # add new row for the person
                df = df.append({'Id': Id, 'Name': name, 'Time In': timeStamp, 'Time Out': ''}, ignore_index=True)
            else: # update the row for the person
                index = df.index[df['Name'] == name][0]
                if df.at[index, 'Time Out'] == '':
                    df.at[index, 'Time Out'] = timeStamp
        
        cv2.imshow('ret',im)
        
        if (cv2.waitKey(1) == ord('q')):
            cam.release()
            cv2.destroyAllWindows()
            break
            
    writer = pd.ExcelWriter('Results.xlsx') # create an Excel writer
    df.to_excel(writer, index=False) # write the DataFrame to the Excel file
    writer.save() # save the Excel file


def start():
    global lmain,cap,lmain,imageFrame
    cap=cv2.VideoCapture(0)
    imageFrame=Frame(width=1500,height=2000,bd=1,bg='green')
    lmain=Label(imageFrame)
    lmain.grid(row=5, column=10)
    imageFrame.place(relx=.3,rely=.2)
    btn_DetectFace=Button(imageFrame,command=lambda:startfac()and generate_timestamp(),text='recognize face',font=('',20),bd=5,width=12).place(relx=.1,rely=.85)
    btn_Stop=Button(imageFrame,command=lambda:stopface(),text='stop recognizing',font=('',20),bd=5,width=12).place(relx=.5,rely=.85)
    
    show_frame()

    
def webcam_screen():
    frm=Frame(win,bg='sky blue')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
    btn_StartCam=Button(command=lambda:start(),text='start camera ',font=('',20,'bold'),bd=10).place(relx=.01,rely=.2)
    btn_StopCam=Button(command=lambda:stop(),text='stop camera',font=('',20,'bold'),bd=10).place(relx=.01,rely=.4)
    btn_back=Button(frm,command=lambda:Welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)
    
def start_capture():
    name = simpledialog.askstring("Input", "What is your name?")
    id_ = simpledialog.askinteger("Input", "What is your ID?")
    if name and id_:
        cam = cv2.VideoCapture(0)
        harcascadePath = "haarcascade_frontalface_default.xml"
        detector = cv2.CascadeClassifier(harcascadePath)
        sampleNum = 0
        count = 0 # Add a count variable
        while True:
            frame, img = cam.read()
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = detector.detectMultiScale(gray, 1.3, 5)
            for (x,y,w,h) in faces:
                #incrementing sample number
                sampleNum = sampleNum + 1
                count += 1 # Increment the count variable
                # Draw a rectangle around the face
                cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
                #saving the captured face in the images folder
                cv2.imwrite(os.path.join("images", f"{id_}.{sampleNum}.jpg"), gray[y:y+h, x:x+w])
            # Display the count variable on the frame
            cv2.putText(img, f"Captured: {count}", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            #display the frame
            cv2.imshow('frame', img)
            #wait for 100 miliseconds 
            if cv2.waitKey(100) & 0xFF == ord('q'):
                break
            #break if the sample number is more than 60
            elif sampleNum > 150:
                break
        cam.release()
        cv2.destroyAllWindows()

        # Append data to Excel file
        now = datetime.datetime.now()
        date_time = now.strftime("%Y-%m-%d %H:%M:%S")
        data = {'Name': [name], 'ID': [id_], 'Date': [date_time]}
        df = pd.DataFrame(data)
        if os.path.exists('Results.xlsx'):
            results_df = pd.read_excel('Results.xlsx')
            results_df = pd.concat([results_df, df], axis=0)
            results_df.to_excel('Results.xlsx', index=False)
        else:
            df.to_excel('Results.xlsx', index=False)

def webcam_screen_capture():
    frm=Frame(win,bg='sky blue')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
    btn_StartCam=Button(command=lambda:start_capture(),text='start camera ',font=('',20,'bold'),bd=10).place(relx=.01,rely=.2)
    btn_StopCam=Button(command=lambda:stop(),text='stop camera',font=('',20,'bold'),bd=10).place(relx=.01,rely=.4)
    btn_back=Button(frm,command=lambda:Welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0) 
    

    
def logout():
    option=messagebox.askyesno('Confirmation','Do you want to logout?')
    if(option==True):
        home_screen()
    else:
        pass
    


def home_screen():
    frm=Frame(win,bg='sky blue')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
    
    lbl_user=Label(frm,text="Username",font=('',20,'bold'),bg='sky blue')
    lbl_user.place(relx=.28,rely=.3)

    entry_user=Entry(frm,font=('',20,'bold'),bd=10)
    entry_user.place(relx=.42,rely=.3)
    entry_user.focus()

    lbl_pass=Label(frm,text="Password",font=('',20,'bold'),bg='sky blue')
    lbl_pass.place(relx=.28,rely=.4)

    entry_pass=Entry(frm,font=('',20,'bold'),bd=10,show="*")
    entry_pass.place(relx=.42,rely=.4)

    btn_forgot_pwd = Button(frm, command=lambda: forgot_password(), text="Forgot Password", font=('', 20, 'bold'), bd=10)
    btn_forgot_pwd.place(relx=.42,rely=.7)
    
    btn_login=Button(frm,command=lambda:welcome_screen(entry_user,entry_pass),text="login",font=('',20,'bold'),bd=10,width=10)
    btn_login.place(relx=.45,rely=.5)
        
    
def predict_screen(entry_user=None,entry_pass=None):
            frm=Frame(win,bg='skyblue')
            frm.place(relx=0,rely=.15,relwidth=1,relheight=1)

            btn_image=Button(frm,command=lambda:image_screen(),text="Use Image",font=('',20,'bold'),bd=10,width=25)
            btn_image.place(relx=.3,rely=.2)

            btn_video=Button(frm,command=lambda:video_screen(),text="Use Video",font=('',20,'bold'),bd=10,width=25)
            btn_video.place(relx=.3,rely=.4)
            
            btn_webcam=Button(frm,command=lambda:webcam_screen(),text="Use Webcam",font=('',20,'bold'),bd=10,width=25)
            btn_webcam.place(relx=.3,rely=.6)

            btn_back=Button(frm,command=lambda:Welcome_screen(),text="back",font=('',20,'bold'),bd=10)
            btn_back.place(relx=.88,rely=0)

def train_model():
    data_dir = "images"
    path = [os.path.join(data_dir, file) for file in os.listdir(data_dir)]

    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split('.')[0])

        faces.append(imageNp)
        ids.append(id)
        cv2.imshow("Training Photos Sample", imageNp)
        cv2.waitKey(1) == 13

    ids = np.array(ids)

    face_recognizer = cv2.face.LBPHFaceRecognizer.create()
    face_recognizer.train(faces, ids)
    face_recognizer.write("classifier.xml")
    cv2.destroyAllWindows()
    messagebox.showinfo("Result", "Training Data Sets have been completed!")

import tkinter as tk
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import os
import cv2
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


def train_accuracy():
   
    data_dir = "test_accuracy"
    path = [os.path.join(data_dir, file) for file in os.listdir(data_dir)]

    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split('.')[0])

        faces.append(imageNp)
        ids.append(id)

    ids = np.array(ids)

    # Split data into training, validation, and test sets
    X_train, X_test, y_train, y_test = train_test_split(faces, ids, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

    # Train the face recognizer on the training set
    face_recognizer = cv2.face.LBPHFaceRecognizer_create()
    face_recognizer.train(X_train, y_train)

    # Evaluate the model on the validation set
    correct_val = 0
    total_val = 0
    for i in range(len(X_val)):
        pred_id, confidence = face_recognizer.predict(X_val[i])
        if pred_id == y_val[i]:
            correct_val += 1
        total_val += 1

    accuracy_val = correct_val / total_val

    # Evaluate the model on the test set
    correct_test = 0
    total_test = 0
    for i in range(len(X_test)):
        pred_id, confidence = face_recognizer.predict(X_test[i])
        if pred_id == y_test[i]:
            correct_test += 1
        total_test += 1

    accuracy_test = correct_test / total_test

    return accuracy_val, accuracy_test


def plot_graph():
    val_accs = []
    test_accs = []

    for i in range(10):
        val_acc, test_acc = train_accuracy()
        val_accs.append(val_acc)
        test_accs.append(test_acc)

    # Create the first figure and plot the validation accuracy
    fig1 = Figure(figsize=(5, 4), dpi=100)
    plot1 = fig1.add_subplot(111)
    plot1.plot(range(1, 11), val_accs, label='Validation Accuracy')
    plot1.set_xlabel('Iteration')
    plot1.set_ylabel('Accuracy')
    plot1.set_title('Validation Accuracy')
    plot1.spines['left'].set_position('zero')
    plot1.legend()

    # Create the second figure and plot the test accuracy
    fig2 = Figure(figsize=(5, 4), dpi=100)
    plot2 = fig2.add_subplot(111)
    plot2.plot(range(1, 11), test_accs, label='Test Accuracy')
    plot2.set_xlabel('Iteration')
    plot2.set_ylabel('Accuracy')
    plot2.set_title('Test Accuracy')
    plot2.spines['left'].set_position('zero')
    plot2.legend()

    # Create the first canvas and display the first figure
    canvas1 = FigureCanvasTkAgg(fig1)
    canvas1.draw()
    canvas1.get_tk_widget().place(relx=0.25, rely=0.5, anchor=tk.CENTER)

    # Create the second canvas and display the second figure
    canvas2 = FigureCanvasTkAgg(fig2)
    canvas2.draw()
    canvas2.get_tk_widget().place(relx=0.75, rely=0.5, anchor=tk.CENTER)






def welcome_screen(entry_user=None, entry_pass=None):
    global user_info  # Use global dictionary for storing username and password

    if entry_user is not None and entry_pass is not None:
        user = entry_user.get()
        pwd = entry_pass.get()
    else:
        user = "admin"
        pwd = "admin"

    if len(user) == 0 or len(pwd) == 0:
        messagebox.showwarning("Validation", "Please fill both fields")
        return
    else:
        if user == user_info["user"] and pwd == user_info["pwd"]:
            frm = Frame(win, bg='skyblue')
            frm.place(relx=0, rely=.15, relwidth=1, relheight=1)

            btn_cap = Button(frm, command=lambda: webcam_screen_capture(), text="Capture Face", font=('', 20, 'bold'),
                             bd=10, width=25)
            btn_cap.place(relx=.3, rely=.25)

            btn_train = Button(frm, command=lambda: train_model(), text="Train Model", font=('', 20, 'bold'),
                               bd=10, width=25)
            btn_train.place(relx=.3, rely=.35)
            
            btn_train = Button(frm, command=lambda: (train_accuracy(),plot_graph()), text="Check training accuracy", font=('', 20, 'bold'),
                               bd=10, width=25)
            btn_train.place(relx=.3, rely=.45)

            btn_wel = Button(frm, command=lambda: predict_screen(), text="Predict", font=('', 20, 'bold'), bd=10,
                             width=25)
            btn_wel.place(relx=.3, rely=.55)

            btn_logout = Button(frm, command=lambda: logout(), text="Logout", font=('', 20, 'bold'), bd=10)
            btn_logout.place(relx=.88, rely=0)

 
            

        else:
            messagebox.showerror("Fail", "Invalid Username/Password")
            
def Welcome_screen(entry_user=None, entry_pass=None):
    

            frm = Frame(win, bg='skyblue')
            frm.place(relx=0, rely=.15, relwidth=1, relheight=1)

            btn_cap = Button(frm, command=lambda: webcam_screen_capture(), text="Capture Face", font=('', 20, 'bold'),
                             bd=10, width=25)
            btn_cap.place(relx=.3, rely=.25)

            btn_train = Button(frm, command=lambda: train_model(), text="Train Model", font=('', 20, 'bold'),
                               bd=10, width=25)
            btn_train.place(relx=.3, rely=.35)
            
            btn_train = Button(frm, command=lambda: train_accuracy() and plot_graph(), text="Check training accuracy", font=('', 20, 'bold'),
                               bd=10, width=25)
            btn_train.place(relx=.3, rely=.45)

            btn_wel = Button(frm, command=lambda: predict_screen(), text="Predict", font=('', 20, 'bold'), bd=10,
                             width=25)
            btn_wel.place(relx=.3, rely=.55)

            btn_logout = Button(frm, command=lambda: logout(), text="Logout", font=('', 20, 'bold'), bd=10)
            btn_logout.place(relx=.88, rely=0)

            
            
user_info = {"user": "admin", "pwd": "admin"}  # Default username and password
security_questions = {"question1": "What is your favorite color?", "question2": "What is your pet's name?"}  # Security questions and answers


def save_user_info():
    with open('user_info.pkl', 'wb') as file:
        pickle.dump(user_info, file)


def load_user_info():
    global user_info
    try:
        with open('user_info.pkl', 'rb') as file:
            user_info = pickle.load(file)
    except FileNotFoundError:
        # If file not found, set default values
        user_info = {"user": "admin", "pwd": "admin"}


def forgot_password():
    global user_info

    # Prompt user for security questions
    answer1 = simpledialog.askstring("Security Question 1", security_questions["question1"], show='*')
    answer2 = simpledialog.askstring("Security Question 2", security_questions["question2"], show='*')

    # Check if answers are correct
    if answer1 == "Red" and answer2 == "Sheru":
        # Prompt user for new password
        new_pwd = simpledialog.askstring("New Password", "Enter new password", show='*')
        if new_pwd is not None:
            # Update password in user_info dictionary
            user_info["pwd"] = new_pwd

            # Save updated user_info dictionary
            save_user_info()

            messagebox.showinfo("Success", "Password updated successfully!")
    else:D
    messagebox.showerror("Fail", "Incorrect answers to security questions")


# Call load_user_info() function at the beginning of the script to load user_info dictionary
load_user_info()
            
home_screen()
win.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "K:\Downloads\Anaconda\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\khush\AppData\Local\Temp\ipykernel_4184\2524183447.py", line 276, in <lambda>
    btn_StopCam=Button(command=lambda:stop(),text='stop camera',font=('',20,'bold'),bd=10).place(relx=.01,rely=.4)
                                      ^^^^^^
  File "C:\Users\khush\AppData\Local\Temp\ipykernel_4184\2524183447.py", line 186, in stop
    lmain.destroy()
  File "K:\Downloads\Anaconda\Lib\tkinter\__init__.py", line 2636, in destroy
    self.tk.call('destroy', self._w)
_tkinter.TclError: can't invoke "destroy" command: application has been destroyed
